# ASGS GeoNames Pipeline

## Required Imports

In [42]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from urllib.request import urlretrieve
from zipfile import ZipFile
import pyogrio

## Required Files

### Create required directories

In [9]:
Path('geonames/').mkdir(exist_ok=True)

Path('asgs/').mkdir(exist_ok=True)

### GeoNames Australian Postcodes

In [20]:
gnames_au_postcodes = 'https://download.geonames.org/export/zip/AU.zip'
urlretrieve(gnames_au_postcodes,'geonames_au_postcodes.zip')

with ZipFile('geonames_au_postcodes.zip','r') as zip:
    zip.extract('AU.txt', path='geonames')

Path(Path.cwd() / 'geonames_au_postcodes.zip').unlink()

### ASGS Files

In [31]:
asgs_geopackages = {'asgs_main.zip': 'https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/ASGS_2021_MAIN_STRUCTURE_GPKG_GDA2020.zip',
                    'asgs_indigenous.zip': 'https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/ASGS_Ed3_2021_Indigenous_Structure_GDA2020_GPKG.zip',
                    'asgs_non_abs.zip': 'https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/ASGS_Ed3_Non_ABS_Structures_GDA2020_updated_2024.zip',
                    'asgs_urban_section_state.zip': 'https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/ASGS_2021_SUA_UCL_SOS_SOSR_GPKG_GDA2020.zip',
                    'asgs_remoteness.zip': 'https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/ASGS_Ed3_2021_RA_GPKG_GDA2020.zip'
                   }

asgs_paths = {}

for zip_name, link in asgs_geopackages.items():
    urlretrieve(link, zip_name)

    with ZipFile(zip_name,'r') as zip:
        for file in zip.namelist():
            if '.gpkg' in file:
                zip.extract(file, path='asgs')
                asgs_paths[zip_name[:-4]] = Path(Path.cwd() / 'asgs' / file)

    Path(Path.cwd() / zip_name).unlink()

In [32]:
asgs_paths

{'asgs_main': PosixPath('/home/hobbymath42/Projects/asgs_geonames_pipeline/asgs/ASGS_2021_Main_Structure_GDA2020.gpkg'),
 'asgs_indigenous': PosixPath('/home/hobbymath42/Projects/asgs_geonames_pipeline/asgs/ASGS_Ed3_2021_Indigenous_Structure_GDA2020.gpkg'),
 'asgs_non_abs': PosixPath('/home/hobbymath42/Projects/asgs_geonames_pipeline/asgs/ASGS_Ed3_Non_ABS_Structures_GDA2020_updated_2024.gpkg'),
 'asgs_urban_section_state': PosixPath('/home/hobbymath42/Projects/asgs_geonames_pipeline/asgs/ASGS_2021_SUA_UCL_SOS_SOSR_GPKG_GDA2020.gpkg'),
 'asgs_remoteness': PosixPath('/home/hobbymath42/Projects/asgs_geonames_pipeline/asgs/ASGS_Ed3_2021_RA_GDA2020.gpkg')}

In [53]:
pyogrio.list_layers(asgs_paths['asgs_main'])

array([['MB_2021_AUST_GDA2020', 'Unknown'],
       ['SA2_2021_AUST_GDA2020', 'Unknown'],
       ['SA1_2021_AUST_GDA2020', 'Unknown'],
       ['SA3_2021_AUST_GDA2020', 'Unknown'],
       ['SA4_2021_AUST_GDA2020', 'Unknown'],
       ['GCCSA_2021_AUST_GDA2020', 'Unknown'],
       ['STE_2021_AUST_GDA2020', 'Unknown'],
       ['AUS_2021_AUST_GDA2020', 'Unknown']], dtype=object)

In [57]:
df = gpd.read_file(asgs_paths['asgs_main'], layer='GCCSA_2021_AUST_GDA2020')

df.query('not(geometry.isna())')

,GCCSA_CODE_2021,GCCSA_NAME_2021,CHANGE_FLAG_2021,CHANGE_LABEL_2021,STATE_CODE_2021,STATE_NAME_2021,AUS_CODE_2021,AUS_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021,geometry
0,1GSYD,Greater Sydney,0,No change,1,New South Wales,AUS,Australia,1.236869e+04,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((151.28159 -33.83318, 151.28167..."
1,1RNSW,Rest of NSW,0,No change,1,New South Wales,AUS,Australia,7.884290e+05,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((159.06230 -31.50886, 159.06244..."
4,2GMEL,Greater Melbourne,0,No change,2,Victoria,AUS,Australia,9.992608e+03,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((144.88829 -38.50260, 144.88826..."
5,2RVIC,Rest of Vic.,0,No change,2,Victoria,AUS,Australia,2.175036e+05,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((146.29286 -39.15778, 146.29275..."
8,3GBRI,Greater Brisbane,0,No change,3,Queensland,AUS,Australia,1.584201e+04,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((153.23213 -27.42089, 153.23179..."
9,3RQLD,Rest of Qld,0,No change,3,Queensland,AUS,Australia,1.714329e+06,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((142.53140 -10.68301, 142.53232..."
12,4GADE,Greater Adelaide,0,No change,4,South Australia,AUS,Australia,3.259886e+03,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((138.65522 -34.55213, 138.65502..."
13,4RSAU,Rest of SA,0,No change,4,South Australia,AUS,Australia,9.809715e+05,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((140.66025 -38.06256, 140.66046..."
16,5GPER,Greater Perth,0,No change,5,Western Australia,AUS,Australia,6.416572e+03,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((115.66990 -32.19610, 115.67003..."
17,5RWAU,Rest of WA,0,No change,5,Western Australia,AUS,Australia,2.520216e+06,http://linked.data.gov.au/dataset/asgsed3/GCCS...,"MULTIPOLYGON (((117.86953 -35.19108, 117.86945..."
